<a href="https://colab.research.google.com/github/TaewonyNet/toys/blob/master/Torrent_To_Google_Drive_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 토렌트 설치 및 구글 드라이브 연결
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
from google.colab import drive
from google.colab import files

# 다운로드 받을 위치
downloadpath = '/Torrent'
mountpath = '/content/drive'
# drive.mount("/content/drive")
drive.mount(mountpath, force_remount=False)

import time
from IPython.display import display
import ipywidgets as widgets
import sys
import requests

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

savepath = mountpath + "/My Drive" + downloadpath

#일반 다운로드용
import threading, requests, time, os
class fileinfo():
    def __init__(self, url, path):
        self.is_init = False
        self.url = url
        self.path = path
        self.current = 0
        self.length = 0
        self.progress = 0
        self.filename = url.split('/')[-1]
        self.requests = None
        self.threading = None
        self.bps = 0
        if not os.path.exists(self.path):
            os.mkdir(self.path)
        pass
    
    def is_complate(self):
        if not self.requests:
            self.is_init = True
            t = threading.Thread(target=self.run)
            t.daemon = True 
            t.start()
            self.threading = t
            return False
        elif self.length == self.current:
            return True
        else:
            return False
            
    def run(self):
        start = time.clock()
        self.requests = requests.get(self.url, stream=True)
        self.requests.raise_for_status()
        self.length = int(self.requests.headers['Content-length'])
        self.current = 0
        with open(os.path.join(self.path, self.filename), 'wb') as f:
            for chunk in self.requests.iter_content(chunk_size=8192): 
                self.current += len(chunk)
                self.progress = self.current / self.length
                done = int(50 * self.progress)
                self.bps = self.current//(time.clock() - start)
#                 sys.stdout.write("\r[%s%s] %s bps" % ('=' * done, ' ' * (50-done), self.current//(time.clock() - start)))
                f.write(chunk)
        self.requests.close()

In [0]:
# 토렌트 파일 선택, 다중 파일 가능
source = files.upload()
for s in list(source.keys()):
    params = {
        "save_path": savepath,
        "ti": lt.torrent_info(s),
    }
    downloads.append(ses.add_torrent(params))
for index, download in enumerate(downloads[:]):
    s = download.status()
    s = str.format("{0} {1}kB/s {2}", download.name(), s.download_rate / 1000, state_str[s.state])
    seed = download.is_seed()
    print(s, seed)

In [0]:
# 마그넷 또는 파일 입력
params = {"save_path": savepath}

while True:
    magnet_link = input("Enter Magnet Link, Download Url Or Enter Empty: ")
    magnet_link_lower = magnet_link.lower()
    if magnet_link_lower == "exit" or not magnet_link:
        break
    if magnet_link_lower.startswith('magnet:?'):
        downloads.append(
            lt.add_magnet_uri(ses, magnet_link, params)
        )
    else:
        downloads.append(
            fileinfo(magnet_link, downloadpath)
        )


In [0]:
# 파일 다운로드 실행
layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if isinstance(download, fileinfo):
            if not download.is_complate():
                bar.description = " ".join(
                    [
                        download.filename,
                        str(download.bps / 1000),
                        "kB/s",
                        'downloading',
                    ]
                )
                bar.value = download.progress * 100
            else:
                next_shift -= 1
                downloads.remove(download)
                bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
                download_bars.remove(bar)
                print(download.filename, "complete")
        else:
            if not download.is_seed():
                s = download.status()

                bar.description = " ".join(
                    [
                        download.name(),
                        str(s.download_rate / 1000),
                        "kB/s",
                        state_str[s.state],
                    ]
                )
                bar.value = s.progress * 100
            else:
                next_shift -= 1
                ses.remove_torrent(download)
                downloads.remove(download)
                bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
                download_bars.remove(bar)
                print(download.name(), "complete")
    time.sleep(1)

In [0]:
from ipywidgets import HTML
HTML("""<a download='test.csv' href='data:text/csv;charset=utf-8;base64,c2VwYWxfbGVuZ3RoLHNlcGFsX3dpZHRoLHBldGFsX2xlbmd0aCxwZXRhbF93aWR0aCxzcGVjaWVzDQo1LjEsMy41LDEuNCwwLjIsc2V0b3NhDQo0LjksMywxLjQsMC4yLHNldG9zYQ0KNC43LDMuMiwxLjMsMC4yLHNldG9zYQ0K'>asdf</a>""")